In [6]:
import ipywidgets as widgets
from IPython.display import display
import subprocess
import psutil
import threading
import time

def open_notebook(s):
    subprocess.Popen(['voila', s])

def on_teacher_clicked(b):
    open_notebook("teacher.ipynb")

def on_student_clicked(b):
    open_notebook("student.ipynb")

def on_terminate_clicked(b):
    selected = listbox.value
    if selected:
        pid = int(selected.split(':')[0])  # Extract PID from the selected option
        try:
            process = psutil.Process(pid)
            process.terminate()  # Terminate the process
            print(f"Terminated process: {selected}")
        except (psutil.NoSuchProcess, psutil.AccessDenied) as e:
            print(f"Error terminating process: {e}")

def get_python_processes_by_ports(port_range):
    processes = []
    for proc in psutil.process_iter(['pid', 'name']):
        try:
            if proc.info['name'] == 'python.exe':  # Filter for python.exe
                for conn in proc.net_connections(kind='inet'):
                    if conn.laddr.port in port_range:
                        processes.append(f"{proc.info['pid']}: {proc.info['name']} (Port: {conn.laddr.port})")
                        break  # No need to check other connections for this process
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            continue
    return processes

# Function to update the listbox periodically
def update_listbox():
    port_range = range(8866, 8878)  # Ports 8866 to 8877
    while True:
        # Get the updated list of python.exe processes using specified ports
        running_processes = get_python_processes_by_ports(port_range)
        listbox.options = running_processes
        time.sleep(2)  # Update every 2 seconds

text = widgets.HTML(
    value = "<h1>Ai Summarize Menu</h1>"
)

student = widgets.Button(
    description = "Student"
)

teacher = widgets.Button(
    description = "Teacher"
)

listbox = widgets.Select(
    description='Running:',
    size=10
)

terminate = widgets.Button(
    description = "Terminate Process"
)

box_layout = widgets.Layout(display='flex',
                    flex_flow='column', 
                    align_items='center',
                    justify_content='center',
                    width='100%',
                    height='100%',
                    margin='250px 0px 0px 0px')

box = widgets.Box(children=[text, student, teacher, listbox, terminate], layout=box_layout)

teacher.on_click(on_teacher_clicked)
student.on_click(on_student_clicked)
terminate.on_click(on_terminate_clicked)
thread = threading.Thread(target=update_listbox, daemon=True)
thread.start()

display(box)

Box(children=(HTML(value='<h1>Ai Summarize Menu</h1>'), Button(description='Student', style=ButtonStyle()), Bu…